# Setup

In [1]:
from game import controlled_run

pygame 2.0.1 (SDL 2.0.14, Python 3.7.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import numpy as np
from game import DO_NOTHING
from game import JUMP

import matplotlib.pyplot as plt

import tensorflow 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

from game import controlled_run

import random
import numpy as np
from collections import deque


# Wrapper

In [3]:
games_count = 0
total_number_of_games = 20

# flaw in game design, it should be based on whether or not you make it past an opponent
class Wrapper(object):
    
    def __init__(self,dqn_solver,games=20):
        self.dqn_solver = dqn_solver
        self.scores=[]
        self.rewards=[]
        self.epoch = 0
        self.games=games
        controlled_run(self,0)
        
    def control(self,values):
        dqn_solver = self.dqn_solver
        if(values['closest_enemy']!=-1):
            state = [[values['closest_enemy']/1000]]
            action = dqn_solver.act(state)
            self.action = action
            self.old_values= values
        else:
            action=0
        return action
    
    def reward(self,old_values,new_values,action,score,done=False):
        import Enemy
        if(old_values['closest_enemy']!= -1):
            state = [[old_values['closest_enemy']/1000]]
            next_state = [[new_values['closest_enemy']/1000]]
            if(new_values['old_closest_enemy']+25 < 380 and new_values['old_closest_enemy']+25 != 24):
                reward = 2
                print("Rewarded for Jumping Over!")
            else:
                reward = 0#new_values['score_increased']
            
            if(done==True):
                reward=-1
                previous =  list(dqn_solver.memory[len(dqn_solver.memory)-1])
                previous[0] = previous[0]
                previous[1] = previous[1]
                previous[2]= reward
                previous[3] = next_state
                previous[4] = done
                previous = tuple(previous)
                dqn_solver.memory = dqn_solver.memory[0:(len(dqn_solver.memory)-1)]
                
                dqn_solver.memory.append(previous)

                # print(previous)

                print("Round: " + str(self.epoch) + " Score: " + str(score))
                # print(dqn_solver.memory[:-min(5,len(dqn_solver.memory))])

                dqn_solver.experience_replay()
            else:
                # print(state,action,reward,next_state,done)
                # print(dqn_solver.memory[:-5])
                dqn_solver.remember(state,action,reward,next_state,done)
                dqn_solver.experience_replay()
    def get_scores(self):
        return self.scores
    
    def get_rewards(self):
        return self.rewards

    def setPlayer(self,player):
        self.player = player

    def setEnemy(self,enemy):
        self.enemy = enemy
    
    def setCurrentScore(self,score):
        self.currentScore = score

    def gameover(self,score,values):
        self.scores.append(score)
        print(self.epoch)
        self.epoch+=1
        if(self.epoch < self.games):
            return controlled_run(self,self.epoch)

# Network

In [4]:
GAMMA = 0.5
ALPHA = 0.5
LR = 0.05
EPSILON = 0.99
EPS_DECAY = 0.95

class DQN:

    def __init__(self, observation_space, action_space):
        
        self.epsilon = EPSILON
        self.alpha = ALPHA
        self.gamma = GAMMA
        self.action_space = action_space
        self.observation_space = observation_space
        
        self.memory = []
        self.batch_size = 128

        # MODEL
        self.model = Sequential()
        self.model.add(Dense(16, input_shape=(observation_space,), kernel_initializer='he_normal'))
        self.model.add(LeakyReLU(0.1))
        self.model.add(Dense(32, kernel_initializer='he_normal'))
        self.model.add(LeakyReLU(0.1))
        self.model.add(Dense(8, kernel_initializer='he_normal'))
        self.model.add(LeakyReLU(0.2))
        self.model.add(Dropout(0.25))
        self.model.add(BatchNormalization())
        self.model.add(Dense(self.action_space, activation="linear"))
        self.model.compile(loss="mse", optimizer=Adam(lr=LR))
        
    def act(self, state):
        if np.random.rand() < self.epsilon:
            # print("RANDOM!")
            return random.randrange(self.action_space)
        q = self.model.predict(state)
        print("Measured: " + str(q[0]), state,np.argmax(q[0]))
        return np.argmax(q[0])

    def experience_replay(self):
        # print(self.memory,self.batch_size)
        import pandas as pd

        mem_sample = random.sample(self.memory, min(len(self.memory),self.batch_size))

        for state, action, reward, next_state, done in mem_sample:
            # print(state, action, reward, next_state, done)
            update_value = reward
            # print(update_value)
            if not done:
                # print("Discounting")
                update_value = self.alpha * (reward + self.gamma * np.max(self.model.predict(next_state)[0]))
            q = self.model.predict(state)
            if(update_value!=0):
                q[0][action] = update_value
            # print(state,type(state))
            # print(state,next_state,action,reward,q,done)
            self.model.train_on_batch(np.array(state), q)
        self.epsilon *= EPS_DECAY
        print("Epsilon: " + str(self.epsilon))

        
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

# Run & Train

In [5]:
dqn_solver = DQN(1,2)
game = Wrapper(dqn_solver,50)

Epsilon: 0.9405
Epsilon: 0.8934749999999999
Epsilon: 0.8488012499999998
Epsilon: 0.8063611874999999
Epsilon: 0.7660431281249999
Measured: [-0.28311187 -0.25482333] [[0.334]] 1
Rewarded for Jumping Over!
Epsilon: 0.7277409717187499
Measured: [-0.48282367  0.11684687] [[0.703]] 1
Epsilon: 0.6913539231328123
Epsilon: 0.6567862269761716
Rewarded for Jumping Over!
Epsilon: 0.623946915627363
Measured: [-0.05660751  0.33735713] [[0.583]] 1
Epsilon: 0.5927495698459949
Rewarded for Jumping Over!
Epsilon: 0.5631120913536951
Epsilon: 0.5349564867860104
Measured: [0.23131731 0.47346258] [[0.541]] 1
Epsilon: 0.5082086624467098
Rewarded for Jumping Over!
Epsilon: 0.4827982293243743
Measured: [-0.0286068   0.37190804] [[0.538]] 1
Epsilon: 0.45865831785815553
Round: 0 Score: 9
Epsilon: 0.4357254019652477
0
Measured: [-0.08084585  0.2665912 ] [[1.0]] 1
Epsilon: 0.41393913186698533
Epsilon: 0.39324217527363603
Measured: [-0.04797763  0.3083499 ] [[1.0]] 1
Epsilon: 0.37358006650995423
Epsilon: 0.35490106

Epsilon: 2.3022108411023178e-05
Measured: [-0.32897842  0.0686423 ] [[1.0]] 1
Epsilon: 2.1871002990472017e-05
Measured: [-0.15815526  0.22103144] [[1.0]] 1
Epsilon: 2.0777452840948416e-05
Measured: [-0.02146944  0.583897  ] [[1.0]] 1
Epsilon: 1.9738580198900995e-05
Measured: [0.70533824 0.8985795 ] [[1.0]] 1
Epsilon: 1.8751651188955945e-05
Measured: [-0.0580525  0.5785559] [[0.976]] 1
Epsilon: 1.7814068629508146e-05
Measured: [-0.25452644 -0.02866372] [[0.667]] 1
Epsilon: 1.6923365198032738e-05
Measured: [-0.12995687  0.08539002] [[0.358]] 1
Epsilon: 1.60771969381311e-05
Measured: [-0.41711247  0.14486806] [[0.904]] 1
Epsilon: 1.5273337091224543e-05
Measured: [-0.04004264  0.2686794 ] [[0.595]] 1
Epsilon: 1.4509670236663315e-05
Measured: [-0.03119142  0.05656039] [[0.286]] 1
Rewarded for Jumping Over!
Epsilon: 1.3784186724830148e-05
Measured: [-0.24434665  0.38765743] [[0.622]] 1
Epsilon: 1.309497738858864e-05
Measured: [0.1272347 0.0023485] [[0.313]] 0
Rewarded for Jumping Over!
Round

KeyboardInterrupt: 

# Results

In [ ]:
scores = game.get_scores()

In [ ]:
plt.plot(scores)
plt.xlabel('Round')
plt.ylabel('Score')
plt.title('Total game score by round')
plt.show()